# Homework

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

## Q1. Downloading the data

We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

    16
    17
    18
    19

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

In [2]:
df = pd.read_parquet("/media/essam/New Volume/Courses/MLOps - Zoomcamp/yellow_tripdata_2023-01.parquet")

In [3]:
df.shape

(3066766, 19)

## Q2. Computing duration

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

    32.59
    42.59
    52.59
    62.59


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [5]:
df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
df['duration'] = df['duration'].dt.total_seconds() / 60

In [6]:
np.std(df['duration'])

42.5943442974141

## Q3. Dropping outliers

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

    90%
    92%
    95%
    98%


In [7]:
# Create a boolean mask for rows to remove
mask = (df['duration'] > 60) | (df['duration'] < 1)

# Calculate percentage of trips that meet the condition
removed_percentage = mask.mean() * 100
print(f"Percentage of trips with duration < 60 minutes or > 1 minute: {100-removed_percentage:.2f}%")
df = df[~mask]

Percentage of trips with duration < 60 minutes or > 1 minute: 98.12%


## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

    Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
    Fit a dictionary vectorizer
    Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

    2
    155
    345
    515
    715


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3009173 entries, 0 to 3066765
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           object        
 8   DOLocationID           object        
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee            floa

In [16]:
dv = DictVectorizer()

df[['PULocationID', 'DOLocationID']] = df[['PULocationID', 'DOLocationID']].astype(str)
data_dicts = df[['PULocationID', 'DOLocationID']].to_dict(orient='records')

X = dv.fit_transform(data_dicts)
print(X.shape[1])

515


## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

    Train a plain linear regression model with default parameters, where duration is the response variable
    Calculate the RMSE of the model on the training data

What's the RMSE on train?

    3.64
    7.64
    11.64
    16.64

In [10]:
target = 'duration'
y_train = df[target].values

In [11]:
lr = LinearRegression()

lr.fit(X, y_train)
y_pred = lr.predict(X)

In [12]:
rmse = root_mean_squared_error(y_train, y_pred)
print(f"RMSE: {rmse:.2f}")

RMSE: 7.65


## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

    3.81
    7.81
    11.81
    16.81

In [13]:
def read_data(filename):
    df = pd.read_parquet(filename)
    df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    df['duration'] = df['duration'].dt.total_seconds() / 60
    mask = (df['duration'] > 60) | (df['duration'] < 1)
    df = df[~mask]
    return df
def prepare_features(df, dv=None):
    df[['PULocationID', 'DOLocationID']] = df[['PULocationID', 'DOLocationID']].astype(str)
    data_dicts = df[['PULocationID', 'DOLocationID']].to_dict(orient='records')
    
    if dv is None:  # Fit only for training
        dv = DictVectorizer(sparse=False)
        X = dv.fit_transform(data_dicts)
        return X, dv
    else:  # Use the same DictVectorizer for validation
        X = dv.transform(data_dicts)
        return X
def prepare_target(df):
    return df['duration'].values
def train_model(X, y):
    lr = LinearRegression()
    lr.fit(X, y)
    return lr


In [ ]:
df_val = read_data("/media/essam/New Volume/Courses/MLOps - Zoomcamp/yellow_tripdata_2023-02.parquet")
val_dicts = df_val[['PULocationID', 'DOLocationID']].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_val = prepare_target(df_val)
y_pred = lr.predict(X_val)
rmse = root_mean_squared_error(y_val, y_pred)
print(f"RMSE: {rmse:.2f}") ## correct answer is 7.81

RMSE: 13.32
